<a href="https://colab.research.google.com/github/dk-wei/ml-algo-implementation/blob/main/%E5%A4%9A%E5%80%BC%E7%A6%BB%E6%95%A3%E7%89%B9%E5%BE%81%E7%9A%84embedding%E8%A7%A3%E5%86%B3%E6%96%B9%E6%A1%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
print(tf.__version__)

1.15.2


# 背景

在处理DeepFM数据时，由于每一个离散特征只有一个取值，因此我们在处理的过程中，将原始数据处理成了两个文件，一个记录特征的索引，一个记录了特征的值，而每一列，则代表一个离散特征。但是如果一个离散特征有多个取值呢？举个例子来说，每个人喜欢的NBA球队，有的人可能喜欢火箭和湖人，有的人可能只喜欢勇士，也有的人喜欢骑士、绿军、猛龙等一大堆。也就是在"喜欢球队“这个特征中，每个人的特征值长度是不同的，也是不定的，对于这种特征，我们本文将其称为多值离散特征。

根据DeepFM的思想，**我们需要将每一个field的特征转换为sparse的，定长的embedding**，即使有多个取值，也是要变换成定长的embedding。那么，一种思路来了，比如一个用户喜欢两个球队，这个field的特征可能是`[1,1,0,0,0,0,0.....0]`，那么我们使用两次embedding lookup，再取个平均不就好了嘛。其实在tensorflow中，有一个函数能够实现我们上述的思路，那就是`tf.nn.embedding_lookup_sparse`。别着急，我们一步一步来实现多值离散特征的embedding处理过程。

`tf.nn.embedding_lookup_sparse`介绍：https://zhuanlan.zhihu.com/p/94212544     
`tf.SparseTensor` 介绍：https://zhuanlan.zhihu.com/p/94035220     
`tf.nn.embedding_lookup`介绍：https://zhuanlan.zhihu.com/p/93846683

# 方法一：使用低阶的tensorflow API

代码来源1：https://blog.csdn.net/qq_38119106/article/details/105399949       
代码来源2：https://blog.csdn.net/zNZQhb07Nr/article/details/109323855

## Input数据

假设我们有三条数据，每条数据代表一个user所喜欢的nba球员，比如有登哥，炮哥，杜老四，慕斯等等：

In [4]:
tags_str=["harden|james|curry",
          "wrestbrook|harden|durant",
          "paul|towns",
    ] #必须是List 每个元素是str

现在建立一个全部球员的集合

In [5]:
TAG_SET = ["harden", "james", "curry", "durant", "paul","towns","wrestbrook"]   #必须是List 每个元素是str


## 数据处理
这里我们需要一个先得到一个`SparseTensor` ，即多为sparse matrix的另一种表示方式，我们只记录非0值所在的位置和值。 

创建`SparseTensor`需要indices, values和dense_shape三个条件。    (详细讲解见附录)

```python
tf.sparse.SparseTensor(
    indices, 
    values, 
    dense_shape
)
```

In [6]:
table = tf.contrib.lookup.index_table_from_tensor(mapping=TAG_SET, default_value=-1)
split_tags = tf.string_split(tags_str,"|")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [7]:
tags=tf.SparseTensor(indices=split_tags.indices,
                     values=table.lookup(split_tags.values),
                     dense_shape=split_tags.dense_shape)

## 为每个tag定义embedding变量

定义我们的embedding parameter. 每个tag都会得到一个长度为3的embedding，本例中总共7个tag，所以embedding的shape为 `7*3`

*注：embedding生成方法随意，可以是random_uniform, 也可以是truncated_normal,自由选择*。

In [16]:
TAG_EMBEDDING_DIM = 3
#embedding_params=tf.Variable(tf.random_uniform([len(TAG_SET), TAG_EMBEDDING_DIM],-1,1))
embedding_params=tf.Variable(tf.truncated_normal([len(TAG_SET), TAG_EMBEDDING_DIM],-1,1))

In [17]:
embedding_params.shape

TensorShape([Dimension(7), Dimension(3)])

## 得到embedding值

将我们刚才得到的`SparseTensor`，传入到`tf.nn.embedding_lookup_sparse`中，结合我们定义的`embedding_params`, 我们就可以得到多值离散特征的embedding值。

3*3的embedding：我们有3个data的embedding，且每个embedding长度为3    

至于每个id的多个`embedding_params`的组合方法可以取均值，也可以用别的算数方法。

In [18]:
embedded_tags=tf.nn.embedding_lookup_sparse(
                                            embedding_params,
                                            sp_ids=tags,
                                            sp_weights=None  # 如果tag的权重很不相同，可以在此初更改
                                            )

`sp_ids`就是我们刚刚得到的`SparseTensor`，而`sp_weights=None`代表的每一个取值的权重，如果是None的话，所有权重都是1，也就是相当于取了平均。如果不是None的话，我们需要同样传入一个`SparseTensor`，代表不同球员的喜欢权重。大家感兴趣可以自己去尝试。

In [21]:
with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(),tf.tables_initializer()])
    a,b=sess.run([embedding_params,embedded_tags])
    print("embedding_params:\n", a)
    print('\n')
    print("embedded_tags:\n", b)

embedding_params:
 [[ 0.75732386 -0.51008767 -0.49978602]
 [-2.432877   -0.86499584 -2.7846866 ]
 [-1.5099285   0.23833692 -1.5290687 ]
 [ 0.25107908 -1.0584562  -0.6477642 ]
 [-2.6425405  -1.0873758  -1.4408138 ]
 [-0.51927495  0.1019839   0.03101754]
 [-1.1071919  -1.6039126  -1.5054555 ]]


embedded_tags:
 [[-1.0618272  -0.37891552 -1.6045138 ]
 [-0.03292966 -1.0574855  -0.8843352 ]
 [-1.5809077  -0.49269593 -0.7048981 ]]


注意的一些问题：
 - TAG_SET是所有可能的特征取值
 - 传入的原特征数据应该是字符类型的list（不知道是否一定是str 必须是list）
 - **tf.tables_initializer()必须初始化**
 - 使用placeholder 我是先传入(n,1)的tensor 然后降维变成list的


# 方法二：使用feature_columns接口

In [41]:
import tensorflow as tf
#from tensorflow.python.feature_column import feature_column
#tf.enable_eager_execution()

# 定义所有商品的集合
TAG_SET = ["harden", "james", "curry", "durant", "paul","towns","wrestbrook"]
# 假设现在有两个用户，用户a和b a点击了a,b,c b点击了a,e,f

PLAYER_DATA = {'players': [['harden', 'james', 'curry'], 
                           ['wrestbrook', 'harden', 'durant'],
                           ['james', 'paul', 'towns']
                           ]}


player_column = tf.feature_column.categorical_column_with_vocabulary_list(
    'players', TAG_SET, dtype=tf.string, default_value=-1
)

embedding_column = tf.feature_column.embedding_column(player_column, 3, combiner='mean')

column_input_layer = tf.feature_column.input_layer(PLAYER_DATA, [embedding_column])
print(column_input_layer)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Tensor("input_layer/concat/concat:0", shape=(3, 3), dtype=float32)


## 附：`SparseTensor`介绍

来源：https://zhuanlan.zhihu.com/p/94035220

1. 先明确一下"sparse"的定义
sparse就是"少"的意思。什么少？数据少。

对应到矩阵来看，sparse matrix就是矩阵中大部分的值都是0，只有少部分值非零。反义词是dense，dense matrix与sparse matrix相反，不再赘述。

2. 那“sparse tensor”又是什么？

sparse matrix太大太复杂了，换一种表现形式，只用几个特性(参数)来描述一个sparse matrix的tensor就叫做sparse tensor。`tf.SparseTensor`就用了三个维度：
  - indices (位置，你想给你的稀疏矩阵的哪个位置放值，就写那个位置。[0,0]就表示矩阵的第0行第0列)
  - value (值，你想给你的稀疏矩阵的这个位置放什么值，就写什么。需要注意的是这里要和上面的indices按照id一一对应)
  - dense_shape (矩阵维度，是你要表示的稀疏矩阵的维度，长*宽)
  
来描述一个sparse matrix。所以sparse matrix和sparse tensor是很容易互相转化的。

举个栗子：



In [42]:
ids = tf.SparseTensor(indices=[[0, 1],
                               [0, 3],
                               [1, 2],
                               [1, 3]],
                      values=[2, 1, 1, 1],
                      dense_shape=[2, 4])

这个sparse tensor表示一个什么sparse matrix呢？

一个2*4的矩阵，矩阵的第(0,1)位置的值是2，第(0,3)位置的值是1，第(1,2)位置的值是1,第(1,3)位置的值是1:
```
[
  [0,2,0,1],
  [0,0,1,1]
]
```

我们来验证一下：

In [49]:
import tensorflow as tf
import numpy as np

# 为了打印
x = tf.sparse_placeholder(tf.float32)
indices = np.array([[0, 1],
                    [0, 3],
                    [1, 2],
                    [1, 3]], dtype=np.int64)
values = np.array([2, 1, 1, 1], dtype=np.int64)
dense_shape = np.array([2, 4], dtype=np.int64)

with tf.Session() as sess:
    # 这么写就是为了打印值
    sparse_tensor = sess.run(x, feed_dict={
        x: tf.SparseTensorValue(indices, values, dense_shape)})
    print('SparseTensor:\n', sparse_tensor)
    tensor_value = tf.sparse_tensor_to_dense(sparse_tensor)
    print('\nSparseMatrix:\n', sess.run(tensor_value))

SparseTensor:
 SparseTensorValue(indices=array([[0, 1],
       [0, 3],
       [1, 2],
       [1, 3]]), values=array([2., 1., 1., 1.], dtype=float32), dense_shape=array([2, 4]))

SparseMatrix:
 [[0. 2. 0. 1.]
 [0. 0. 1. 1.]]
